In [31]:
import requests
import pandas as pd
from nba_api.live.nba.endpoints import scoreboard
from nba_api.live.nba.endpoints import boxscore
from datetime import datetime

In [3]:
board = scoreboard.ScoreBoard()
games = board.games.get_dict()

In [11]:
game_date = board.score_board_date

In [16]:
games[1].keys()

dict_keys(['gameId', 'gameCode', 'gameStatus', 'gameStatusText', 'period', 'gameClock', 'gameTimeUTC', 'gameEt', 'regulationPeriods', 'ifNecessary', 'seriesGameNumber', 'seriesText', 'homeTeam', 'awayTeam', 'gameLeaders', 'pbOdds'])

In [23]:
games[1]['awayTeam'].keys()

dict_keys(['teamId', 'teamName', 'teamCity', 'teamTricode', 'wins', 'losses', 'score', 'seed', 'inBonus', 'timeoutsRemaining', 'periods'])

In [69]:
id_list = []
print('Games Taking Place Today - {}'.format(game_date))

for game in games:
    id = game['gameId']
    away = game['awayTeam']['teamCity']+" "+game['awayTeam']['teamName']
    home = game['homeTeam']['teamCity']+" "+game['homeTeam']['teamName']
    start = datetime.strptime(game['gameEt'], "%Y-%m-%dT%H:%M:%S%z")
    id_list.append(id)
    print('Game ID {}: {} at {} - {} start'.format(id, away, home, start.strftime("%H:%M")))

Games Taking Place Today - 2022-01-14
Game ID 0022100632: Orlando Magic at Charlotte Hornets - 19:00 start
Game ID 0022100633: Toronto Raptors at Detroit Pistons - 19:00 start
Game ID 0022100634: Phoenix Suns at Indiana Pacers - 19:00 start
Game ID 0022100635: Boston Celtics at Philadelphia 76ers - 19:00 start
Game ID 0022100637: Golden State Warriors at Chicago Bulls - 19:30 start
Game ID 0022100636: Atlanta Hawks at Miami Heat - 20:00 start
Game ID 0022100638: Cleveland Cavaliers at San Antonio Spurs - 20:30 start
Game ID 0022100639: Dallas Mavericks at Memphis Grizzlies - 22:00 start
Game ID 0022100640: Houston Rockets at Sacramento Kings - 22:00 start


In [71]:
box = boxscore.BoxScore(id_list[1])

In [146]:
box.away_team_player_stats.get_dict

<bound method Endpoint.DataSet.get_dict of <nba_api.live.nba.endpoints._base.Endpoint.DataSet object at 0x00000113AC4B7FD0>>

In [142]:
box_df = pd.DataFrame(box.away_team_player_stats.get_dict())

In [143]:
box_df_combined = box_df.join(pd.DataFrame((box_df.pop('statistics').values.tolist())))

In [145]:
print(box_df_combined)

      status  order  personId jerseyNum position starter oncourt played  \
0     ACTIVE      1   1628384         3       SF       1       0      1   
1     ACTIVE      2   1627783        43       PF       1       0      1   
2     ACTIVE      3    203920        24        C       1       0      1   
3     ACTIVE      4   1630567         4       SG       1       0      1   
4     ACTIVE      5   1627832        23       PG       1       0      1   
5     ACTIVE      6   1630173         5      NaN       0       0      1   
6     ACTIVE      7   1628449        25      NaN       0       1      1   
7     ACTIVE      8   1630551        11      NaN       0       0      1   
8     ACTIVE      9   1629139        18      NaN       0       1      1   
9     ACTIVE     10   1629004        14      NaN       0       1      1   
10    ACTIVE     11   1630625        45      NaN       0       1      1   
11    ACTIVE     12   1630201        22      NaN       0       1      1   
12  INACTIVE     13   162

In [97]:
players_list = []
away_stats = box.away_team_player_stats.get_dict()
home_stats = box.away_team_player_stats.get_dict()
print(len(away_stats))
print(len(home_stats))
#box.home_team_player_stats.get_dict()

17
17


In [101]:
away_stats[0]['name']

'OG Anunoby'

In [168]:
daily_stats = []

for i in id_list:
    box = boxscore.BoxScore(i)
    away_df = pd.DataFrame(box.away_team_player_stats.get_dict())
    home_df = pd.DataFrame(box.home_team_player_stats.get_dict())
    away_df_stats = away_df.join(pd.DataFrame((away_df.pop('statistics').values.tolist())))
    home_df_stats = home_df.join(pd.DataFrame((home_df.pop('statistics').values.tolist())))
    daily_stats.append(away_df_stats)
    daily_stats.append(home_df_stats)
    
daily_stats_df = pd.concat(daily_stats)

In [169]:
for col in daily_stats_df.columns:
    print(col)

status
order
personId
jerseyNum
position
starter
oncourt
played
name
nameI
firstName
familyName
notPlayingReason
notPlayingDescription
assists
blocks
blocksReceived
fieldGoalsAttempted
fieldGoalsMade
fieldGoalsPercentage
foulsOffensive
foulsDrawn
foulsPersonal
foulsTechnical
freeThrowsAttempted
freeThrowsMade
freeThrowsPercentage
minus
minutes
minutesCalculated
plus
plusMinusPoints
points
pointsFastBreak
pointsInThePaint
pointsSecondChance
reboundsDefensive
reboundsOffensive
reboundsTotal
steals
threePointersAttempted
threePointersMade
threePointersPercentage
turnovers
twoPointersAttempted
twoPointersMade
twoPointersPercentage


In [178]:
stat_categories = ['jerseyNum','position','name', 'minutes', 'points', 'reboundsTotal','assists','steals','blocks']

In [ ]:
sort_by = ['points']

In [ ]:
asc_list = ['false']

In [180]:
pd.set_option("display.max_rows", 500)

In [181]:
daily_stats_df[stat_categories].sort_values('points',ascending=False)

,jerseyNum,position,name,minutes,points,reboundsTotal,assists,steals,blocks
3,1,SG,Devin Booker,PT35M16.94S,35,4,3,0,1
4,10,PG,Darius Garland,PT37M21.02S,32,4,8,1,1
4,5,PG,Dejounte Murray,PT35M37.98S,30,14,8,0,0
4,5,PG,De'Aaron Fox,PT34M27.00S,27,6,8,2,0
2,22,C,Deandre Ayton,PT29M56.00S,27,12,1,2,1
4,77,PG,Luka Doncic,PT31M45.00S,27,12,10,0,0
1,35,PF,Marvin Bagley III,PT32M51.98S,26,13,1,0,2
2,35,C,Christian Wood,PT26M22.00S,26,8,0,2,1
8,21,NaN,Moritz Wagner,PT27M46.00S,26,4,1,1,1
0,8,SF,Justin Holiday,PT36M55.00S,25,2,4,2,3
